In [1]:
!pwd

/home/jovyan/work/notebooks


In [2]:
!spark-submit --version

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 2.4.5
      /_/
                        
Using Scala version 2.11.12, OpenJDK 64-Bit Server VM, 1.8.0_252
Branch HEAD
Compiled by user centos on 2020-02-02T19:38:06Z
Revision cee4ecbb16917fa85f02c635925e2687400aa56b
Url https://gitbox.apache.org/repos/asf/spark.git
Type --help for more information.


In [2]:
from pyspark.sql import SparkSession

# Spark session & context
spark = (SparkSession
         .builder
         .master('local')
         #.config("spark.jars", "/home/jovyan/work/notebooks/jar-files/spark-streaming-kafka-0-10_2.12-2.4.3.jar")
         #.config("spark.jars", "/home/jovyan/work/notebooks/jar-files/spark-sql-kafka-0-10_2.11-2.4.5.jar")
         .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.5")
         .getOrCreate())
sc = spark.sparkContext

In [12]:
df = (spark
  .readStream
  .format("kafka")
  .option("kafka.bootstrap.servers", "kafka-server:9092")
  .option("subscribe", "wiki-changes")
  .option("startingOffsets", "earliest")
  .load())

DataFrame[key: string, value: string]

In [31]:
df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [33]:
df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

AnalysisException: 'Queries with streaming sources must be executed with writeStream.start();;\nkafka'

In [17]:
queryStream =(
    df
    .writeStream
    .format("console")
    .outputMode("append")
    .start())

In [19]:
queryStream.stop()

In [21]:
queryStreamMem =(
    df
    .writeStream
    .format("memory")
    .queryName("wiki_changes")
    .outputMode("append")
    .start())

In [28]:
queryStreamMem.stop()

In [30]:
spark.sql("select * from wiki_changes").show()

+----+--------------------+------------+---------+------+--------------------+-------------+
| key|               value|       topic|partition|offset|           timestamp|timestampType|
+----+--------------------+------------+---------+------+--------------------+-------------+
|null|[7B 22 24 73 63 6...|wiki-changes|        0|     0|2020-06-27 09:44:...|            0|
|null|[7B 22 24 73 63 6...|wiki-changes|        0|     1|2020-06-27 09:44:...|            0|
|null|[7B 22 24 73 63 6...|wiki-changes|        0|     2|2020-06-27 09:44:...|            0|
|null|[7B 22 24 73 63 6...|wiki-changes|        0|     3|2020-06-27 09:44:...|            0|
|null|[7B 22 24 73 63 6...|wiki-changes|        0|     4|2020-06-27 09:44:...|            0|
|null|[7B 22 24 73 63 6...|wiki-changes|        0|     5|2020-06-27 09:44:...|            0|
|null|[7B 22 24 73 63 6...|wiki-changes|        0|     6|2020-06-27 09:44:...|            0|
|null|[7B 22 24 73 63 6...|wiki-changes|        0|     7|2020-06-27 09